In [47]:
import sys
from glob import glob
from collections import defaultdict

sys.path.append("/scratch/group/csce435-f23/python-3.8.17/lib/python3.8/site-packages")
sys.path.append("/scratch/group/csce435-f23/thicket")

import pandas as pd

import thicket as th

In [48]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# POINT THIS AT YOUR DATA

In [159]:
# Point this at the top directory of all your cali files for all of the implementations. Other files can be in the directory too, that is ok.
FILES_LOCATION = "Thicket/cuda_selection/"

# Reader check
Can the files be read in one-by-one

In [160]:
working_files = []
error_files = []
i = 0
team_files = glob(f"{FILES_LOCATION}/**/*.cali", recursive=True)
for file in team_files:
    try:
        tk = th.Thicket.from_caliperreader(file)
        working_files.append(file)
    except Exception:
        i += 1
        error_files.append(file)

In [156]:
print("Files that could be read in individually (one-by-one):")
print(f"{len(working_files)}/{len(team_files)} ({len(working_files)/len(team_files)*100}%)")

Files that could be read in individually (one-by-one):
156/156 (100.0%)


# Check for Metadata columns

Check for the necessary metadata columns from the [report](https://github.com/TAMU-CSCE435-Pearce/Project/blob/master/Report.md#3b-collect-metadata)

In [157]:
team_metadata_valid = []
metadata_columns=['cali.caliper.version', 'spot.options', 'spot.channels', 'cali.channel',
    'launchdate',
    'libraries', 'cmdline', 'cluster', 'Algorithm', 'ProgrammingModel',
    'Datatype', 'SizeOfDatatype', 'InputSize', 'InputType',
    'group_num', 'implementation_source']
mpi_cols = ['num_procs']
cuda_cols = ['num_threads', 'num_blocks',]
metadata_col_dict = defaultdict(lambda: [])

team_files = glob(f"{FILES_LOCATION}/**/*.cali", recursive=True)
for file in team_files:
    try:
        valid = True
        tk = th.Thicket.from_caliperreader(file)
        cols = tk.metadata.columns
        model_to_check = []
        if "CUDA" in tk.metadata["ProgrammingModel"].to_list()[0].upper():
            model_to_check = metadata_columns + cuda_cols
        else:
            model_to_check = metadata_columns + mpi_cols
        for col in model_to_check:
            if col not in cols:
                metadata_col_dict[list(tk.profile_mapping.values())[0]].append(col)
                valid=False
        if valid:
            team_metadata_valid.append(file)
    except KeyError:
        pass

for file, cols in metadata_col_dict.items():
    print(f"File '{file}' missing metadata columns:\n\t{cols}")

# Check for DataFrame columns

Check for the necessary DataFrame columns from the [report](https://github.com/TAMU-CSCE435-Pearce/Project/blob/master/Report.md#4c-you-should-measure-the-following-performance-metrics). For the GPU columns, you need one or the other column in the tuple, not both.

In [163]:
team_dataframe_valid = []
necessary_columns = ["Min time/rank","Max time/rank","Avg time/rank","Total time",]
not_gpu_columns = ["Variance time/rank",]
gpu_columns=[("Avg GPU time/rank", "Avg GPU Time/rank"),
             ("Min GPU time/rank", "Min GPU Time/rank"),
             ("Max GPU time/rank", "Max GPU Time/rank"),
             ("Total GPU time", "Total GPU Time"),]
def check_df_cols(tk, dict):
    valid = True
    cols = tk.dataframe.columns
    for col in necessary_columns:
        if col not in cols:
            dict[list(tk.profile_mapping.values())[0]].append(col)
            valid = False
    if "ProgrammingModel" in tk.metadata.columns:
        if "CUDA" in tk.metadata["ProgrammingModel"].to_list()[0].upper():
            for col in gpu_columns:
                if col[0] not in cols and col[1] not in cols:
                    dict[list(tk.profile_mapping.values())[0]].append(col)
                    valid = False
        else:
            for col in not_gpu_columns:
                if col not in cols:
                    dict[list(tk.profile_mapping.values())[0]].append(col)
                    valid = False
    return valid

dataframe_col_dict = defaultdict(lambda: [])
team_files = glob(f"{FILES_LOCATION}/**/*.cali", recursive=True)
for file in team_files:
    tk = th.Thicket.from_caliperreader(file)
    valid = check_df_cols(tk, dataframe_col_dict)
    if valid:
        team_dataframe_valid.append(file)

print(f"Number of files missing columns: {len(dataframe_col_dict.items())}")
for file, cols in dataframe_col_dict.items():
    print(f"File '{file}' missing dataframe columns: {cols}")

Number of files missing columns: 36
File 'Thicket/cuda_selection\strong\nearly\CUDA-nearly-t1024-v16777216.cali' missing dataframe columns: [('Avg GPU time/rank', 'Avg GPU Time/rank'), ('Min GPU time/rank', 'Min GPU Time/rank'), ('Max GPU time/rank', 'Max GPU Time/rank'), ('Total GPU time', 'Total GPU Time')]
File 'Thicket/cuda_selection\strong\nearly\CUDA-nearly-t128-v16777216.cali' missing dataframe columns: [('Avg GPU time/rank', 'Avg GPU Time/rank'), ('Min GPU time/rank', 'Min GPU Time/rank'), ('Max GPU time/rank', 'Max GPU Time/rank'), ('Total GPU time', 'Total GPU Time')]
File 'Thicket/cuda_selection\strong\nearly\CUDA-nearly-t256-v16777216.cali' missing dataframe columns: [('Avg GPU time/rank', 'Avg GPU Time/rank'), ('Min GPU time/rank', 'Min GPU Time/rank'), ('Max GPU time/rank', 'Max GPU Time/rank'), ('Total GPU time', 'Total GPU Time')]
File 'Thicket/cuda_selection\strong\nearly\CUDA-nearly-t512-v16777216.cali' missing dataframe columns: [('Avg GPU time/rank', 'Avg GPU Time/r

# Try all files together

In [104]:
tk = th.Thicket.from_caliperreader(team_files)

# Check tree

Should be no different from the [report](https://github.com/TAMU-CSCE435-Pearce/Project/blob/master/Report.md#3a-caliper-instrumentation), spelling and all.

In [105]:
tk.statsframe.dataframe["time"] = 1
print(tk.tree())

  _____ _     _      _        _   
 |_   _| |__ (_) ___| | _____| |_ 
   | | | '_ \| |/ __| |/ / _ \ __|
   | | | | | | | (__|   <  __/ |_ 
   |_| |_| |_|_|\___|_|\_\___|\__|  v2023.3.0

1.000 main
├─ 1.000 comm
│  └─ 1.000 comm_large
│     ├─ 1.000 comp_d2h
│     ├─ 1.000 comp_h2d
│     ├─ 1.000 correctness_d2h
│     ├─ 1.000 correctness_h2d
│     ├─ 1.000 data_init_d2h
│     └─ 1.000 data_init_h2d
├─ 1.000 comp
│  └─ 1.000 comp_large
├─ 1.000 correctness_check
└─ 1.000 data_init

Legend (Metric: time Min: 1.00 Max: 1.00)
█ 1.00 - 1.00
█ 1.00 - 1.00
█ 1.00 - 1.00
█ 1.00 - 1.00
█ 1.00 - 1.00
█ 1.00 - 1.00

name User code    ◀  Only in left graph    ▶  Only in right graph



In [106]:
# Groupby programming model. Should result in 2 thickets, MPI and CUDA.
gb_pmodel = tk.groupby("ProgrammingModel")

1  thickets created...
{'CUDA': <thicket.thicket.Thicket object at 0x0000017BD2071E40>}


In [107]:
# Groupby the parameters we ran with. After this operation, each Thicket in gb_total should contain profiles with unique InputSizes (there should be no duplicate input sizes).
gb_cuda = gb_pmodel["CUDA"].groupby(["ProgrammingModel", "Algorithm", "InputType", "num_threads"])
# gb_mpi = gb_pmodel["MPI"].groupby(["ProgrammingModel", "Algorithm", "InputType", "num_procs"])
# gb_total = {**gb_cuda, **gb_mpi}

gb_total = {**gb_cuda}

20  thickets created...
{('CUDA', 'Odd-Even Bubble Sort', 'nearly', 64): <thicket.thicket.Thicket object at 0x0000017BD4CFA320>, ('CUDA', 'Odd-Even Bubble Sort', 'nearly', 128): <thicket.thicket.Thicket object at 0x0000017BD4ECE7D0>, ('CUDA', 'Odd-Even Bubble Sort', 'nearly', 256): <thicket.thicket.Thicket object at 0x0000017BD207EE00>, ('CUDA', 'Odd-Even Bubble Sort', 'nearly', 512): <thicket.thicket.Thicket object at 0x0000017BD1FD7400>, ('CUDA', 'Odd-Even Bubble Sort', 'nearly', 1024): <thicket.thicket.Thicket object at 0x0000017BD47DAC50>, ('CUDA', 'Odd-Even Bubble Sort', 'random', 64): <thicket.thicket.Thicket object at 0x0000017BD0098310>, ('CUDA', 'Odd-Even Bubble Sort', 'random', 128): <thicket.thicket.Thicket object at 0x0000017BD1B81B70>, ('CUDA', 'Odd-Even Bubble Sort', 'random', 256): <thicket.thicket.Thicket object at 0x0000017BD2BB2E90>, ('CUDA', 'Odd-Even Bubble Sort', 'random', 512): <thicket.thicket.Thicket object at 0x0000017BCE2F40D0>, ('CUDA', 'Odd-Even Bubble Sort'

In [108]:
# Compose all of the data back together. If this step errors, you probably have duplicate inputsizes. Run 1a to check for this.
ctk = th.Thicket.concat_thickets(
    thickets=list(gb_total.values()),
    axis="columns",
    headers=list(gb_total.keys()),
    metadata_key="InputSize"
)

In [109]:
ctk.dataframe.head(50)

(CUDA, Odd-Even Bubble Sort, nearly, 64)   
                                                                                                 nid   
node                                              InputSize                                            
{'name': 'main', 'type': 'function'}              65536                                            1  \
                                                  1048576                                          1   
                                                  16777216                                         1   
{'name': 'comm', 'type': 'function'}              65536                                            2   
                                                  1048576                                          2   
                                                  16777216                                         2   
{'name': 'comm_large', 'type': 'function'}        65536                                            3   
                                                  1048576                                          3   
                                                  16777216                                         3   
{'name': 'comp_d2h', 'type': 'function'}          65536                                           10   
                                                  1048576                                         10   
                                                  16777216                                        10   
{'name': 'comp_h2d', 'type': 'function'}          65536                                            7   
                                                  1048576                                          7   
                                                  16777216                                         7   
{'name': 'correctness_d2h', 'type': 'function'}   65536                                           13   
                                                  1048576                                         13   
                                                  16777216                                        13   
{'name': 'correctness_h2d', 'type': 'function'}   65536                                           11   
                                                  1048576                                         11   
                                                  16777216                                        11   
{'name': 'data_init_d2h', 'type': 'function'}     65536                                            6   
                                                  1048576                                          6   
                                                  16777216                                         6   
{'name': 'data_init_h2d', 'type': 'function'}     65536                                            4   
                                                  1048576                                          4   
                                                  16777216                                         4   
{'name': 'comp', 'type': 'function'}              65536                                            8   
                                                  1048576                                          8   
                                                  16777216                                         8   
{'name': 'comp_large', 'type': 'function'}        65536                                            9   
                                                  1048576                                          9   
                                                  16777216                                         9   
{'name': 'correctness_check', 'type': 'function'} 65536                                           12   
                                                  1048576                                         12   
                                                  16777216                                        12   
{'na

# 1A

Check for duplicate input sizes

In [110]:
i = 0
for key in list(gb_total.keys()):
    print(i)
    print(gb_total[key].profile_mapping)
    print(gb_total[key].metadata["InputSize"])
    i += 1

0
OrderedDict([(1500438313, 'Thicket/bubble/cuda_bubble/strong\\nearly\\Bubble-CUDA-nearly-t64-v65536.cali'), (1768037032, 'Thicket/bubble/cuda_bubble/strong\\nearly\\Bubble-CUDA-nearly-t64-v16777216.cali'), (3272417618, 'Thicket/bubble/cuda_bubble/strong\\nearly\\Bubble-CUDA-nearly-t64-v1048576.cali')])
profile
1500438313       65536
1768037032    16777216
3272417618     1048576
Name: InputSize, dtype: int64
1
OrderedDict([(806976117, 'Thicket/bubble/cuda_bubble/strong\\nearly\\Bubble-CUDA-nearly-t128-v16777216.cali'), (1238830872, 'Thicket/bubble/cuda_bubble/strong\\nearly\\Bubble-CUDA-nearly-t128-v65536.cali'), (3744315234, 'Thicket/bubble/cuda_bubble/strong\\nearly\\Bubble-CUDA-nearly-t128-v1048576.cali')])
profile
806976117     16777216
1238830872       65536
3744315234     1048576
Name: InputSize, dtype: int64
2
OrderedDict([(523087446, 'Thicket/bubble/cuda_bubble/strong\\nearly\\Bubble-CUDA-nearly-t256-v16777216.cali'), (1021443304, 'Thicket/bubble/cuda_bubble/strong\\nearly\\Bu